In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
from pathlib import Path

import matplotlib.pyplot as plt

plt.style.use("ggplot")

params = {
    "text.color": (0.25, 0.25, 0.25),
    "figure.figsize": [16, 9],
}

plt.rcParams.update(params)

# Get colors from default theme.
DEFAULT_COLORS = plt.rcParams["axes.prop_cycle"].by_key()["color"]

import pandas as pd
import seaborn as sns

# Auto-detect the most recent CSV file
csv_files = sorted(Path(".").glob("github_activity_*.csv"), reverse=True)
CSV_FILE = csv_files[0] if csv_files else "github_activity.csv"
print(f"Using data file: {CSV_FILE}")

## 1. Load and Explore the Data


In [ ]:
# Load the data
df = pd.read_csv(CSV_FILE)
print(df.info(verbose=True))

In [ ]:
# Preview the data
df.head(5)

In [ ]:
# Parse date columns
date_cols = ["created_at", "updated_at", "closed_at", "author_date"]
for col in date_cols:
    df[col] = pd.to_datetime(df[col], errors="coerce", utc=True)

# Create a unified date column (author_date for commits, created_at for others)
df["activity_date"] = df["author_date"].fillna(df["created_at"])

# Extract date components
df["year"] = df["activity_date"].dt.year
df["month"] = df["activity_date"].dt.month
df["month_name"] = df["activity_date"].dt.month_name()
df["week"] = df["activity_date"].dt.isocalendar().week
df["day_of_week"] = df["activity_date"].dt.day_name()
df["hour"] = df["activity_date"].dt.hour

print("Date range:")
print(f"  From: {df['activity_date'].min()}")
print(f"  To:   {df['activity_date'].max()}")

## 2. Activity Overview


In [ ]:
# Activity types breakdown
activity_counts = df["kind"].value_counts()
print("Activity Types:")
print(activity_counts)
print(f"\nTotal activities: {len(df):,}")

In [ ]:
# Visualize activity types
if not activity_counts.empty:
    fig, ax = plt.subplots(figsize=(16, 6))
    sns.barplot(x=activity_counts.values, y=activity_counts.index, ax=ax)
    ax.set_xlabel("Count")
    ax.set_title("GitHub Activity Types Distribution")
    plt.tight_layout()
    plt.show()
else:
    print("No activity data to visualize.")

## 3. Repository Analysis


In [ ]:
# Focus on commits for repository analysis
commits_df = df[df["kind"] == "commits"].copy()
print(f"Total commits: {len(commits_df):,}")

if len(commits_df) == 0:
    print("No commits found in the data.")
else:
    # Top repositories by commit count
    top_n = 25
    repo_commits = commits_df["repo"].value_counts().head(top_n)
    print(f"\nTop {top_n} Repositories by Commits:")
    print(repo_commits)

In [ ]:
# Visualize top repositories
if len(commits_df) > 0:
    fig, ax = plt.subplots(figsize=(16, 16))
    colors = sns.color_palette("viridis", len(repo_commits))

    # Clean repo names for display
    repo_names = [r.split("/")[-1] for r in repo_commits.index]

    bars = ax.barh(repo_names[::-1], repo_commits.values[::-1], color=colors[::-1])
    ax.set_xlabel("Number of Commits")
    ax.set_title(f"Top {top_n} Repositories by Commit Count")

    # Add value labels
    for bar, val in zip(bars, repo_commits.values[::-1], strict=False):
        ax.text(val + 1, bar.get_y() + bar.get_height() / 2, f"{val}", va="center", fontsize=9)

    plt.tight_layout()
    plt.show()
else:
    print("No commits to visualize.")

In [ ]:
# Organization breakdown
org_activity = df["org"].value_counts()
print("Activity by Organization:")
print(org_activity)

## 4. Temporal Analysis


In [ ]:
# Monthly commit trend
monthly_commits = commits_df.groupby(
    commits_df["activity_date"].dt.tz_localize(None).dt.to_period("M").astype(str)
).size()

fig, ax = plt.subplots(figsize=(16, 6))
sns.barplot(x=monthly_commits.index, y=monthly_commits.values, ax=ax, color=DEFAULT_COLORS[1])
ax.set_xlabel("Month")
ax.set_ylabel("Number of Commits")
ax.set_title("Monthly Commit Activity")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Day of week activity pattern
day_order = [
    "Monday",
    "Tuesday",
    "Wednesday",
    "Thursday",
    "Friday",
    "Saturday",
    "Sunday",
]
day_commits = commits_df["day_of_week"].value_counts().reindex(day_order)

fig, ax = plt.subplots(figsize=(16, 6))
colors = [
    DEFAULT_COLORS[0] if d in ["Saturday", "Sunday"] else DEFAULT_COLORS[1] for d in day_order
]
sns.barplot(
    x=day_commits.index,
    y=day_commits.values,
    hue=day_commits.index,
    palette=colors,
    legend=False,
    ax=ax,
)
ax.set_xlabel("Day of Week")
ax.set_ylabel("Number of Commits")
ax.set_title("Commits by Day of Week (Weekend in green)")

# Add value labels
for i, val in enumerate(day_commits.values):
    ax.text(i, val + 2, f"{val}", ha="center", fontsize=10)

plt.tight_layout()
plt.show()

# Weekend vs weekday stats
weekend_commits = day_commits[["Saturday", "Sunday"]].sum()
weekday_commits = day_commits[["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]].sum()
print(f"Weekday commits: {weekday_commits} ({weekday_commits / len(commits_df) * 100:.1f}%)")
print(f"Weekend commits: {weekend_commits} ({weekend_commits / len(commits_df) * 100:.1f}%)")

In [ ]:
# Hourly activity pattern
hour_commits = commits_df["hour"].value_counts().reindex(range(24), fill_value=0).reset_index()
hour_commits.columns = ["hour", "count"]

fig, ax = plt.subplots(figsize=(16, 6))
sns.barplot(
    data=hour_commits, x="hour", y="count", hue="hour", palette="viridis", legend=False, ax=ax
)
ax.set_xlabel("Hour of Day (UTC)")
ax.set_ylabel("Number of Commits")
ax.set_title("Commits by Hour of Day")

plt.tight_layout()
plt.show()

# Peak hours
peak_hours = hour_commits.nlargest(3, "count")
print(f"\nPeak commit hours (UTC): {peak_hours['hour'].tolist()}")

In [ ]:
# Heatmap: Day of week vs Hour
heatmap_data = commits_df.groupby(["day_of_week", "hour"]).size().unstack(fill_value=0)
heatmap_data = heatmap_data.reindex(day_order)

fig, ax = plt.subplots(figsize=(16, 6))
sns.heatmap(
    heatmap_data,
    cmap="YlOrRd",
    annot=False,
    fmt="d",
    ax=ax,
    cbar_kws={"label": "Number of Commits"},
)
ax.set_xlabel("Hour of Day (UTC)")
ax.set_ylabel("Day of Week")
ax.set_title("Commit Activity Heatmap: Day of Week vs Hour")
plt.tight_layout()
plt.show()

## 5. Commit Message Analysis


In [ ]:
# Analyze commit messages
commits_df["message_length"] = commits_df["message"].str.len()
commits_df["word_count"] = commits_df["message"].str.split().str.len()

print("Commit Message Statistics:")
print(f"  Average length: {commits_df['message_length'].mean():.1f} characters")
print(f"  Median length: {commits_df['message_length'].median():.1f} characters")
print(f"  Average word count: {commits_df['word_count'].mean():.1f} words")

In [ ]:
# Common commit message patterns
message_patterns = {
    "Update": commits_df["message"].str.contains("Update", case=False, na=False).sum(),
    "Fix": commits_df["message"].str.contains("Fix", case=False, na=False).sum(),
    "Add": commits_df["message"].str.contains("Add", case=False, na=False).sum(),
    "Initial": commits_df["message"].str.contains("Initial", case=False, na=False).sum(),
    "Refactor": commits_df["message"].str.contains("Refactor", case=False, na=False).sum(),
    "Merge": commits_df["message"].str.contains("Merge", case=False, na=False).sum(),
    "README": commits_df["message"].str.contains("README", case=False, na=False).sum(),
    "Intermediate": commits_df["message"].str.contains("Intermediate", case=False, na=False).sum(),
    "Housekeeping": commits_df["message"].str.contains("Housekeeping", case=False, na=False).sum(),
}

patterns_df = pd.Series(message_patterns).sort_values(ascending=False)

fig, ax = plt.subplots(figsize=(16, 6))
bars = ax.barh(patterns_df.index[::-1], patterns_df.values[::-1])
ax.set_xlabel("Count")
ax.set_title("Common Commit Message Keywords")

plt.tight_layout()
plt.show()

In [ ]:
# Sample of commit messages
print("Sample Commit Messages:")
print(commits_df[["repo", "message"]].sample(10).to_string(index=False))

## 6. Project Focus Analysis


In [ ]:
# Identify project themes from repository names
repos = commits_df["repo"].unique()
print(f"Total unique repositories: {len(repos)}")
print("\nAll repositories (sorted by commit count):")
repo_counts = commits_df["repo"].value_counts()
max_count_width = len(str(repo_counts.max()))
for repo, count in repo_counts.items():
    print(f"  {count:>{max_count_width}}  {repo.split('/')[-1]}")

In [ ]:
# Activity timeline per top repository
top_n = 25
top_repos = repo_commits.head(top_n).index.tolist()

# Prepare data for seaborn
timeline_data = []
for repo in top_repos:
    repo_data = commits_df[commits_df["repo"] == repo].copy()
    # Remove timezone before converting to period to avoid warning
    monthly = (
        repo_data.groupby(repo_data["activity_date"].dt.tz_localize(None).dt.to_period("M"))
        .size()
        .reset_index()
    )
    monthly.columns = ["month", "commits"]
    monthly["month"] = monthly["month"].dt.to_timestamp()
    monthly["repo"] = repo.split("/")[-1]
    timeline_data.append(monthly)

timeline_df = pd.concat(timeline_data, ignore_index=True)

# Create faceted line plots
g = sns.FacetGrid(timeline_df, row="repo", height=2.5, aspect=4, sharey=False)
g.map_dataframe(sns.lineplot, x="month", y="commits", marker="o", color=DEFAULT_COLORS[1])
g.set_axis_labels("", "")
g.set_titles(row_template="{row_name}")
g.fig.suptitle("Monthly Commit Activity - Top 5 Repositories", y=1.02, fontsize=14)

# Set x-axis to show all months with proper formatting
import matplotlib.dates as mdates

for ax in g.axes.flat:
    ax.xaxis.set_major_locator(mdates.MonthLocator())
    ax.xaxis.set_major_formatter(mdates.DateFormatter("%b %Y"))
    ax.tick_params(axis="x", rotation=45)

plt.tight_layout()
plt.show()

## 7. Pull Requests & Issues Analysis


In [ ]:
# Analyze non-commit activities
non_commits = df[df["kind"] != "commits"].copy()
print(f"Non-commit activities: {len(non_commits)}")
print("\nBreakdown:")
print(non_commits["kind"].value_counts())

In [ ]:
# Issues and PRs details
if len(non_commits) > 0:
    print("Pull Requests and Issues:")
    display_cols = ["kind", "repo", "title", "state", "created_at"]
    available_cols = [c for c in display_cols if c in non_commits.columns]
    print(non_commits[available_cols].to_string(index=False))

## 8. Summary Statistics


In [ ]:
# Calculate summary statistics
if len(commits_df) == 0:
    print("No commits found - cannot generate summary statistics.")
else:
    total_days = (commits_df["activity_date"].max() - commits_df["activity_date"].min()).days
    active_days = commits_df["activity_date"].dt.date.nunique()

    print("=" * 60)
    print("GITHUB ACTIVITY SUMMARY")
    print("=" * 60)
    print(
        f"\n📅 Time Period: {commits_df['activity_date'].min().strftime('%Y-%m-%d')} to {commits_df['activity_date'].max().strftime('%Y-%m-%d')}"
    )
    print(f"   Total span: {total_days} days")
    print(f"   Active days: {active_days} days ({active_days / total_days * 100:.1f}% of period)")

    print("\n📊 Activity Counts:")
    print(f"   Total commits: {len(commits_df):,}")
    print(f"   Unique repositories: {commits_df['repo'].nunique()}")
    print(f"   Organizations: {commits_df['org'].nunique()}")

    print("\n📈 Productivity Metrics:")
    print(f"   Commits per day (average): {len(commits_df) / total_days:.2f}")
    print(f"   Commits per active day: {len(commits_df) / active_days:.2f}")
    print(f"   Commits per week (average): {len(commits_df) / (total_days / 7):.2f}")

    print("\n🏆 Most Active Repository:")
    top_repo = repo_commits.idxmax()
    print(f"   {top_repo.split('/')[-1]}: {repo_commits.max()} commits")

    print("\n⏰ Peak Activity:")
    peak_month = monthly_commits.idxmax()
    peak_hour_idx = hour_commits["count"].idxmax()
    peak_hour = hour_commits.loc[peak_hour_idx, "hour"]
    peak_hour_count = hour_commits.loc[peak_hour_idx, "count"]
    print(f"   Most active month: {peak_month} ({monthly_commits.max()} commits)")
    print(f"   Most active day: {day_commits.idxmax()} ({day_commits.max()} commits)")
    print(f"   Most active hour: {peak_hour}:00 UTC ({peak_hour_count} commits)")

    print("\n" + "=" * 60)